<a href="https://colab.research.google.com/github/Sarvagya4/Banking77/blob/main/Banking_77_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch] datasets pandas scikit-learn wandb evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.7/357.7 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import pandas as pd
import numpy as np
from getpass import getpass

import torch
import wandb
import evaluate
from datasets import Dataset, DatasetDict

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_DIR = "/content/drive/MyDrive/Banking77_Project"
except ImportError:
    BASE_DIR = "./Banking77_Project"
    print("Not in Google Colab. Using local directory for project files.")

DATA_DIR = os.path.join(BASE_DIR, "data")
MODEL_DIR = os.path.join(BASE_DIR, "models", "bert-full-finetune")
os.makedirs(MODEL_DIR, exist_ok=True)

print(f"Project directory: {BASE_DIR}")
print(f"Data directory: {DATA_DIR}")
print(f"Model will be saved in: {MODEL_DIR}")

Mounted at /content/drive
Project directory: /content/drive/MyDrive/Banking77_Project
Data directory: /content/drive/MyDrive/Banking77_Project/data
Model will be saved in: /content/drive/MyDrive/Banking77_Project/models/bert-full-finetune


In [ ]:
try:
    wandb_key = getpass("Enter your Weights & Biases API key: ")
    wandb.login(key=wandb_key)
except Exception as e:
    print(f"Could not log in to W&B. Please check your API key. Error: {e}")

Enter your Weights & Biases API key: ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sarvagyatayal (Banking77) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
train_df = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
val_df = pd.read_csv(os.path.join(DATA_DIR, 'validation.csv'))
test_df = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [ ]:
id2label = {i: label for i, label in enumerate(train_df['intent'].astype('category').cat.categories)}
label2id = {label: i for i, label in id2label.items()}
NUM_LABELS = len(id2label)

In [ ]:
train_df.rename(columns={'intent': 'label'}, inplace=True)
val_df.rename(columns={'intent': 'label'}, inplace=True)
test_df.rename(columns={'intent': 'label'}, inplace=True)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

print("Datasets loaded successfully:")
print(dataset_dict)

Datasets loaded successfully:
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'text_cleaned'],
        num_rows: 8002
    })
    validation: Dataset({
        features: ['text', 'label', 'text_cleaned'],
        num_rows: 2001
    })
    test: Dataset({
        features: ['text', 'label', 'text_cleaned'],
        num_rows: 3080
    })
})


In [ ]:
run = wandb.init(
    project="Banking77-Intent-Classification",
    job_type="train",
    name="day2-bert-full-finetune",
    notes="Fine-tuning a standard BERT model on the Banking77 dataset."
)

In [ ]:
MODEL_NAME = "bert-base-uncased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    # Use the 'text_cleaned' column created in Day 1.
    return tokenizer(examples["text_cleaned"], padding="max_length", truncation=True)

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)
print("\nDatasets tokenized:")
print(tokenized_datasets)

Map:   0%|          | 0/8002 [00:00<?, ? examples/s]

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3080 [00:00<?, ? examples/s]


Datasets tokenized:
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'text_cleaned', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8002
    })
    validation: Dataset({
        features: ['text', 'label', 'text_cleaned', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2001
    })
    test: Dataset({
        features: ['text', 'label', 'text_cleaned', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3080
    })
})


In [ ]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")

    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"],
    }

In [ ]:
training_args = TrainingArguments(
    output_dir=MODEL_DIR,
    num_train_epochs=6,  # A common starting point for fine-tuning
    learning_rate=1e-5,  # A standard learning rate for BERT fine-tuning[14]
    lr_scheduler_type="linear",  # or "cosine"
    warmup_ratio=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    eval_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",        # Save a checkpoint at the end of each epoch
    load_best_model_at_end=True,  # The best model will be loaded at the end of training
    metric_for_best_model="f1",
    logging_dir=f"{MODEL_DIR}/logs",
    logging_steps=100,
    fp16=True,
    report_to="wandb"  # This is crucial for W&B integration[18]
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-3029316986.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: fp16 mixed precision requires a GPU (not 'xla').

In [ ]:
print("Starting model training...")
trainer.train()
print("Training finished.")

Starting model training...


NameError: name 'trainer' is not defined